## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-01-20-50-27 +0000,nyt,Trump Becomes the Wild Card in Razor-Thin Hond...,https://www.nytimes.com/2025/12/01/world/ameri...
1,2025-12-01-20-50-19 +0000,nyt,Zelensky Turns to Europe After Talks With U.S....,https://www.nytimes.com/2025/12/01/world/europ...
2,2025-12-01-20-48-34 +0000,wapo,Conservatives virtually tied in Honduras elect...,https://www.washingtonpost.com/world/2025/11/3...
3,2025-12-01-20-42-56 +0000,nypost,Jeffrey Epstein name-dropped at Luigi Mangione...,https://nypost.com/2025/12/01/us-news/jeffrey-...
4,2025-12-01-20-40-17 +0000,nypost,Schumer says his New York offices were under b...,https://nypost.com/2025/12/01/us-news/schumer-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2329/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,39
27,new,17
52,house,13
501,up,11
51,white,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
43,2025-12-01-19-15-59 +0000,bbc,Appeals court disqualifies ex-Trump lawyer Ali...,https://www.bbc.com/news/articles/cewjg272dq9o...,86
118,2025-12-01-14-42-17 +0000,nypost,Appeals court disqualifies former Trump lawyer...,https://nypost.com/2025/12/01/us-news/appeals-...,85
14,2025-12-01-20-17-00 +0000,wsj,The White House said Trump underwent advanced ...,https://www.wsj.com/politics/white-house-says-...,85
46,2025-12-01-19-00-50 +0000,nypost,Trump rails against ‘AOC of Tennessee’ in elec...,https://nypost.com/2025/12/01/us-news/presiden...,82
97,2025-12-01-16-20-00 +0000,wsj,Court Upholds Disqualification of Trump’s Top ...,https://www.wsj.com/us-news/law/court-upholds-...,82


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
43,86,2025-12-01-19-15-59 +0000,bbc,Appeals court disqualifies ex-Trump lawyer Ali...,https://www.bbc.com/news/articles/cewjg272dq9o...
7,51,2025-12-01-20-31-23 +0000,nypost,"Bannon, top conservatives urge White House to ...",https://nypost.com/2025/12/01/business/bannon-...
23,43,2025-12-01-19-56-53 +0000,bbc,Three children among four killed in shooting a...,https://www.bbc.com/news/articles/cvg81p61dnmo...
231,43,2025-11-30-23-47-00 +0000,wsj,Congress launched inquiries and lawmakers from...,https://www.wsj.com/politics/national-security...
181,40,2025-12-01-09-57-37 +0000,cbc,"Death toll from floods in Indonesia, Sri Lanka...",https://www.cbc.ca/news/world/death-toll-from-...
238,37,2025-11-30-22-28-52 +0000,nypost,Why Venezuela’s allies Russia and China are sl...,https://nypost.com/2025/11/30/world-news/why-v...
48,35,2025-12-01-18-53-03 +0000,nypost,Starbucks to pay $35M to NYC workers to settle...,https://nypost.com/2025/12/01/business/starbuc...
26,33,2025-12-01-19-47-57 +0000,nypost,Luigi Mangione’s bushy eyebrows ‘only thing’ e...,https://nypost.com/2025/12/01/us-news/luigi-ma...
223,31,2025-12-01-01-00-00 +0000,wsj,"More than other generations, 20-somethings are...",https://www.wsj.com/economy/consumers/gen-z-sh...
46,30,2025-12-01-19-00-50 +0000,nypost,Trump rails against ‘AOC of Tennessee’ in elec...,https://nypost.com/2025/12/01/us-news/presiden...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
